In [6]:
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
from biogeme.expressions import (
    Beta,
    Variable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
)

In [7]:
np.random.seed(seed=90267)
df = pd.read_csv('F:/more work/netherlands/wave8data/model/nested_logit/nested_data_4mode.csv')


In [8]:
df.drop(['AFSTV', 'G_AFSTAND', 'KAFSTV', 'KHVM', 'KVERTTIJD', 'REISDUUR', 'KREISDUUR', 'REISDUUR_ORG', 'G_REISDUUR', 'ROLAUTO', 'PARKEERKOSTEN', 'KRVM', 'Bestuurder', 'HHBRUTOINK2_w5','CarStatus' , 'TransitStatus'] , axis=1 , inplace=True)
df.dropna(inplace=True)
df=df[df["PROV_27"]==1]
df["ID"] = df["ID"]*0.0001
df["CarTravelTime"] = df["CarTravelTime"]*0.0001
df["BikeDistance"] = df["BikeDistance"]*0.00001
df["TransitDistance"] = df["TransitDistance"]*0.00001
df["TransitDuration"] = df["TransitDuration"]*0.0001
df["alokatie_station"] = df["alokatie_station"]*0.0001
df["alokatie_bushalte2xpu"] = df["alokatie_bushalte2xpu"]*0.0001
df["vlokatie_centrumsgw"] = df["vlokatie_centrumsgw"]*0.0001

database = db.Database('first_logit_model', df)
globals().update(database.variables)



In [9]:
# Parameters to be estimated
#car as a driver
ASC_car = Beta('ASC_car', 0, None, None, 0)
N_ACTIVITEIT13_1_car = Beta('N_ACTIVITEIT13_1_car', 0, None, None, 0)
#WERKSITUATIE_ANDERS1_car = Beta('WERKSITUATIE_ANDERS1_car', 0, None, None, 0)
vlokatie_centrumsgw_car = Beta('vlokatie_centrumsgw_car', 0, None, None, 0)
KAFSTV_3_car = Beta('KAFSTV_3_car', 0, None, None, 0)
#GEBRUIK_PARK2_car = Beta('GEBRUIK_PARK2_car', 0, None, None, 0)
KLEEFT2_9_car = Beta('KLEEFT2_9_car', 0, None, None, 0)
INVLOED3_car = Beta('INVLOED3_car', 0, None, None, 0)
DOEL_8_car = Beta('DOEL_8_car', 0, None, None, 0)
N_KIND_2_car = Beta('N_KIND_2_car', 0, None, None, 0)
##EVENT4_car = Beta('EVENT4_car', 0, None, None, 0)





# train
ASC_bus_Train = Beta('ASC_bus_Train', 0, None, None, 1)
#TransitDuration_bus_Train = Beta('TransitDuration_bus_Train', 0, None, None, 0)
CarTravelTime_bus_Train = Beta('CarTravelTime_bus_Train', 0, None, None, 0)
#KAFSTV_12_bus_Train = Beta('KAFSTV_12_bus_Train', 0, None, None, 0)
PAUTO_bus_Train = Beta('PAUTO_bus_Train', 0, None, None, 0)
MOTIEF_11_bus_Train = Beta('MOTIEF_11_bus_Train', 0, None, None, 0)
KAFSTV_12_bus_Train = Beta('KAFSTV_12_bus_Train', 0, None, None, 0)

#N_KIND






# bicycle
ASC_Scooter_Bicycle = Beta('ASC_Bicycle', 0, None, None, 0)
MOTIEF_5_Bicycle = Beta('MOTIEF_5_Bicycle', 0, None, None, 0)
TYPE_OVKAART4_Bicycle = Beta('TYPE_OVKAART4_Bicycle', 0, None, None, 0)
KLEEFT2_8_Bicycle = Beta('KLEEFT2_8_Bicycle', 0, None, None, 0)
KAFSTV_2_Bicycle = Beta('KAFSTV_2_Bicycle', 0, None, None, 0)
#EVENT9_Bicycle = Beta('EVENT9_Bicycle', 0, None, None, 0)
TYPE_OVKAART2_Bicycle = Beta('TYPE_OVKAART2_Bicycle', 0, None, None, 0)
EVENT6_Bicycle = Beta('EVENT6_Bicycle', 0, None, None, 0)
new_Bicycle = Beta('new_Bicycle', 0, None, None, 0)









# walking
ASC_Walking = Beta('ASC_Walking', 0, None, None, 0)
TransitDuration_Walking = Beta('TransitDuration_Walking', 0, None, None, 0)
HERKOMST_w5_2_Walking = Beta('HERKOMST_w5_2_Walking', 0, None, None, 0)
new_mixed = Beta('new_mixed', 0, None, None, 0)
new_mixed2 = Beta('new_mixed2', 0, None, None, 0)



In [10]:

# Define a random parameter, normally distributed, designed to be used
# for Monte-Carlo simulation
#EVENT5_m = Beta('EVENT5_m', 1, None, None, 0)

# It is advised not to use 0 as starting value for the following parameter.
vlokatie_centrumsgw_S = Beta('vlokatie_centrumsgw_S', 1, None, None, 0)
#vlokatie_centrumsgw_S_RND = vlokatie_centrumsgw_car + N_ACTIVITEIT13_1_car*N_ACTIVITEIT13_1 + vlokatie_centrumsgw_S * bioDraws('vlokatie_centrumsgw_S_RND', 'NORMAL')WERKSITUATIE_ANDERS1+ROLAUTO_2
vlokatie_centrumsgw_S_RND = vlokatie_centrumsgw_car + new_mixed*MOTIEF_7 + new_mixed2*ROLAUTO_2 +  vlokatie_centrumsgw_S * bioDraws('vlokatie_centrumsgw_S_RND', 'NORMAL')

In [11]:
# Definition of the utility functions
V1 = ASC_car + vlokatie_centrumsgw_S_RND*vlokatie_centrumsgw + KAFSTV_3_car*KAFSTV_3  

V2 = ASC_bus_Train + PAUTO_bus_Train*PAUTO + KAFSTV_12_bus_Train*KAFSTV_12

V3 = ASC_Scooter_Bicycle + MOTIEF_5_Bicycle*MOTIEF_5 + KLEEFT2_8_Bicycle*KLEEFT2_8 + KAFSTV_2_Bicycle*KAFSTV_2

V4 = ASC_Walking + HERKOMST_w5_2_Walking*HERKOMST_w5_2 + TransitDuration_Walking*TransitDuration

In [12]:
# Associate utility functions with the numbering of alternatives
V = {1: V1, 2: V2, 3: V3, 4: V4}
# Associate the availability conditions with the alternatives
av = {1: av_car_as_driver, 2: av_train, 3: av_Bicycle , 4: av_Walking}

In [13]:
obsprob = models.logit(V, av, Choice_modals)

# Conditional to the random parameters, the likelihood of all observations for
# one individual (the trajectory) is the product of the likelihood of
# each observation.
# We integrate over B_TIME_RND using Monte-Carlo
logprob = log(MonteCarlo(obsprob))


In [14]:
# Define level of verbosity
#logger = msg.bioMessage()
# logger.setSilent()
# logger.setWarning()
# logger.setGeneral()
#logger.setDetailed()
# logger.setDebug()

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob, numberOfDraws=1000)
biogeme.modelName = 'mlogit_no_pannel_8'

# Estimate the parameters.
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(pandasResults)


The use of argument numberOfDraws in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_draws in section MonteCarloof the .toml parameter file. The default file name is biogeme.toml


                            Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_Bicycle             -1.706747      0.202284    -8.437376      0.000000
ASC_Walking              0.849635      0.190087     4.469724      0.000008
ASC_car                 -3.706349      0.776071    -4.775787      0.000002
HERKOMST_w5_2_Walking   -1.213771      0.493886    -2.457592      0.013987
KAFSTV_12_bus_Train      0.667545      0.307058     2.174005      0.029705
KAFSTV_2_Bicycle         1.951819      0.587043     3.324833      0.000885
KAFSTV_3_car             2.024553      0.708975     2.855605      0.004295
KLEEFT2_8_Bicycle        1.074320      0.480244     2.237030      0.025284
MOTIEF_5_Bicycle         2.402511      0.665110     3.612201      0.000304
PAUTO_bus_Train         -0.484281      0.218058    -2.220885      0.026359
TransitDuration_Walking -1.410898      0.483389    -2.918763      0.003514
new_mixed                2.627528      1.777603     1.478129      0.139373
new_mixed2               

In [15]:
print(results.shortSummary())
print(results.getEstimatedParameters(onlyRobust=False))

Results for model mlogit_no_pannel_8
Nbr of parameters:		15
Sample size:			563
Excluded data:			0
Final log likelihood:		-550.4326
Akaike Information Criterion:	1130.865
Bayesian Information Criterion:	1195.864

                            Value   Std err    t-test       p-value  \
ASC_Bicycle             -1.706747  0.197226 -8.653764  0.000000e+00   
ASC_Walking              0.849635  0.186021  4.567424  4.937544e-06   
ASC_car                 -3.706349  0.673971 -5.499274  3.813586e-08   
HERKOMST_w5_2_Walking   -1.213771  0.499152 -2.431664  1.502965e-02   
KAFSTV_12_bus_Train      0.667545  0.302623  2.205867  2.739331e-02   
KAFSTV_2_Bicycle         1.951819  0.584425  3.339727  8.386093e-04   
KAFSTV_3_car             2.024553  0.869835  2.327514  1.993790e-02   
KLEEFT2_8_Bicycle        1.074320  0.555696  1.933287  5.320091e-02   
MOTIEF_5_Bicycle         2.402511  0.746588  3.217988  1.290933e-03   
PAUTO_bus_Train         -0.484281  0.210815 -2.297187  2.160810e-02   
Transit